In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import numpy as np
import os
import random
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pickle
from functions.space import Space
from torch.optim.lr_scheduler import StepLR
import optuna 
from indices_selection_splitting import split_dataset_stratified

seed_value = 42
torch.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# Run first the script 'Dimensionality_reduction\dimensionality_reduction_module.ipynb' to generate the encoding and decoding of the data

In [ ]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")
def create_path(name):
    if not os.path.exists(name):
        os.makedirs(name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Study and storage settings
study_name = 'autoencoder_GCN_optimization_1'
storage_url = f'sqlite:///{study_name}.db'

test_name = 'Opt_GCN_v1'

dim_red_dir = os.path.join('Dimensionality_reduction', 'output')
dataset_dir = os.path.join('Dataset', 'agard_dataset.npy')
grid_data_dir = os.path.join('Dataset', 'grid_data')

output_dir = os.path.join('output', test_name)
model_dir = os.path.join('Model_architecture', test_name)
model_filename = "best_model.pth"

create_path(output_dir)
create_path(model_dir)



# Transfer Learning: True-> loads the stored weights and trains from there; False-> trains from new
Transfer_Learning_flag = True
# Training Flag: True-> model.fit is executed and saved; False-> not trained neither stored
Execute_Train_flag = True

#############################
# Wing geometry specs
xref = 0.4637/2    # Reference point for the x-coordinate (half of the chord length)
chord = 0.4637     # Chord length of the wing
span = 0.75        # Span of the wing
area = 0.353       # Reference area of the wing
#############################

In [ ]:
# Load dataset
dataset = np.load(dataset_dir) # Shape: (251, 45943, 13) - (samples, nodes, features) # features: [x, y, z, CP, CFx, CFy, CFz, mode1, mode2, mode3, mode4, mode5, mode6]

X = dataset[:, :, np.concatenate((np.arange(3), np.arange(7, dataset.shape[2])))]
coordinates = X[:,:,:3]
Y = dataset[:,:,3:7] # Output
print( '-> Shape of the input matrix: ',X.shape)
print( '-> Shape of the output matrix: ',Y.shape)

In [ ]:
## Feature normalisation:
scaler = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = X.shape
X = np.reshape(X, newshape=(-1, variables))
X = scaler.fit_transform(X)
X = np.reshape(X, newshape=(samples, points, variables))

scalery = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = Y.shape
Y = np.reshape(Y, newshape=(-1, variables))
Y = scalery.fit_transform(Y)
Y = np.reshape(Y, newshape=(samples, points, variables))

In [ ]:
train_indices, val_indices, test_indices = split_dataset_stratified(X, Y, 
                                                                    k=20,
                                                                    train_ratio=0.65,
                                                                    val_ratio=0.25,
                                                                    test_ratio=0.15,
                                                                    )  # Split the dataset into train, val, and test indices using stratified sampling

X_train, X_val, X_test, Y_train, Y_val, Y_test = X[train_indices], X[val_indices], X[test_indices], Y[train_indices], Y[val_indices], Y[test_indices]  # Create train, val, and test splits for X and Y

# Create the dictionary
indices_dict = {
    'train_indices': train_indices,  # Store train indices
    'val_indices': val_indices,      # Store validation indices
    'test_indices': test_indices     # Store test indices
}

print(f"Train len: {len(train_indices)}")  
print(f"Val len: {len(val_indices)}")      
print(f"Test len: {len(test_indices)}")    
print(f"Tot len: {len(train_indices)+len(val_indices)+len(test_indices)}")  

# Save the dictionary to a pickle file
with open('indices.pkl', 'wb') as f:
    pickle.dump(indices_dict, f)  # Save indices dictionary to a pickle file

print("Indices saved to indices.pkl")  # Confirm that indices have been saved

np.random.seed(seed_value)  # Set numpy random seed for reproducibility
random.seed(seed_value)     # Set python random seed for reproducibility

In [ ]:
# This is the code for loading the variables:

space_data = []

for sample in range(dataset.shape[0]):
    print(f"{sample+1}/{dataset.shape[0]}", end="\r")  # Print progress for each sample
    output_dim_red_dir = dim_red_dir + f'Sample_{sample:03d}\\'  # Build path to the sample's directory
    sp1_path = output_dim_red_dir + "space1"  # Path to space1 file
    sp2_path = output_dim_red_dir + "space2"  # Path to space2 file
    space1 = Space()  # Create Space object for space1
    space2 = Space()  # Create Space object for space2
    space1.load(sp1_path)  # Load data into space1 from file
    space2.load(sp2_path)  # Load data into space2 from file
    data = (space1, space2)  # Tuple containing both space objects
    space_data.append(data)  # Append tuple to space_data list


In [ ]:
grid_data = []

for i in range(dataset.shape[0]):  
    grid_data_file = torch.tensor(np.load(os.path.join(grid_data_dir, f'grid_data_{i:03d}.npy')), dtype=torch.float32).to(device)  # Load grid data for sample i and move to device
    cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal = grid_data_file[:,0], grid_data_file[:,1], grid_data_file[:,2], grid_data_file[:,3]  # Extract relevant columns
    coordinates_tensor = torch.tensor(coordinates[i,:,:], dtype=torch.float32).to(device)  # Convert coordinates for sample i to tensor and move to device
    grid_data.append((cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, coordinates_tensor))  # Append tuple to grid_data list


In [ ]:
# Define a custom PyTorch Geometric Dataset

class CustomGraphData(Data):
    def __init__(self, x, y, space, grid_data):
        """
        Custom data object for graph data, extending torch_geometric.data.Data.

        Args:
            x (Tensor): Node features.
            y (Tensor): Target values.
            space (tuple): Tuple of Space objects for graph structure.
            grid_data (tuple): Tuple containing grid-related tensors.
        """
        super().__init__(x, y)
        self.x = x  # Node features
        self.y = y  # Target values
        self.space = space  # Space objects for graph structure
        self.grid_data = grid_data  # Grid-related tensors

class CustomGraphDataset(Dataset):
    def __init__(self, X, Y, space, grid_data):
        """
        Custom dataset for graph data.

        Args:
            X (np.ndarray): Input features.
            Y (np.ndarray): Target values.
            space (list): List of Space tuples.
            grid_data (list): List of grid data tuples.
        """
        if not (len(X) == len(Y) and len(Y) == len(space)):
            raise ValueError("X Y and space must be the same length")  # Ensure all inputs have the same length
        self.X = X  # Input features
        self.Y = Y  # Target values
        self.space = space  # Space objects
        self.grid_data = grid_data  # Grid data

    def __len__(self):
        """Return the number of samples in the dataset."""
        return len(self.X)

    def __getitem__(self, idx):
        """
        Get a single sample from the dataset.

        Args:
            idx (int): Index of the sample.

        Returns:
            CustomGraphData: Data object containing features, targets, space, and grid data.
        """
        x = torch.tensor(self.X[idx], dtype=torch.float32)  # Convert input features to tensor
        y = torch.tensor(self.Y[idx], dtype=torch.float32)  # Convert target values to tensor
        space = self.space[idx]  # Get space tuple
        grid_data = self.grid_data[idx]  # Get grid data tuple
        data = CustomGraphData(x=x, y=y, space=space, grid_data=grid_data)  # Create data object
        return data

# Create instances of the custom Dataset for train, val, and test splits
space_data_train = [space_data[i] for i in train_indices]  # Training space data
space_data_val = [space_data[i] for i in val_indices]      # Validation space data
space_data_test = [space_data[i] for i in test_indices]    # Test space data

grid_data_train = [grid_data[i] for i in train_indices]    # Training grid data
grid_data_val = [grid_data[i] for i in val_indices]        # Validation grid data
grid_data_test = [grid_data[i] for i in test_indices]      # Test grid data

train_dataset = CustomGraphDataset(X_train, Y_train, space_data_train, grid_data_train)  # Training dataset
val_dataset = CustomGraphDataset(X_val, Y_val, space_data_val, grid_data_val)           # Validation dataset
test_dataset = CustomGraphDataset(X_test, Y_test, space_data_test, grid_data_test)       # Test dataset

def collate(data_list):
    """
    Custom collate function to move data to the appropriate device.

    Args:
        data_list (list): List of CustomGraphData objects.

    Returns:
        list: List of data objects moved to device if CUDA is available.
    """
    if torch.cuda.is_available():
        data_list = [data.to(device) for data in data_list]  # Move each data object to device
    return data_list

batch_size = 1  # Set batch size

def seed_worker(worker_id):
    """
    Seed worker function for reproducibility in DataLoader workers.

    Args:
        worker_id (int): Worker ID.
    """
    worker_seed = torch.initial_seed() % 2**32  # Generate worker seed
    np.random.seed(worker_seed)  # Seed numpy
    random.seed(worker_seed)     # Seed random

g = torch.Generator()  # Create a torch generator
g.manual_seed(seed_value)  # Set manual seed for reproducibility

# Create DataLoaders for train, validation, and test sets
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    worker_init_fn=seed_worker, generator=g, collate_fn=collate
)  # Training DataLoader

val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True,
    worker_init_fn=seed_worker, generator=g, collate_fn=collate
)  # Validation DataLoader

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    worker_init_fn=seed_worker, generator=g, collate_fn=collate
)  # Test DataLoader

total_samples = len(train_loader.dataset)  # Total samples in training set
batch_size = train_loader.batch_size       # Batch size used
num_batches = len(train_loader)            # Number of batches in training set

print(f"Total number of samples in train_loader: {total_samples}")  # Print total samples
print(f"Batch size in train_loader: {batch_size}")                  # Print batch size
print(f"Number of batches in train_loader: {num_batches}")          # Print number of batches

batch = next(iter(train_loader))  # Get the first batch
print("Batch:", batch)            # Print the batch

In [ ]:
class GraphConv(torch.nn.Module):
    """
    Graph Convolutional Layer using GCNConv with optional output activation.
    """
    def __init__(self, in_channels, out_channels, output=False):
        super(GraphConv, self).__init__()
        self.conv = GCNConv(in_channels, out_channels, add_self_loops=True, improved=True, cached=True)  # GCN layer
        self.prelu = nn.PReLU()  # PReLU activation
        self.output = output  # Whether this is an output layer

    def forward(self, x, edge_indices, edge_distances):
        """
        Forward pass for GraphConv.
        Args:
            x: Node features.
            edge_indices: Edge indices.
            edge_distances: Edge weights.
        Returns:
            Output tensor after convolution and activation.
        """
        x = self.conv(x, edge_index=edge_indices, edge_weight=edge_distances)  # Apply GCN
        if not self.output:
            x = self.prelu(x)  # Apply activation if not output
        return x

class Encoder(torch.nn.Module):
    """
    Encoder module using sparse matrix multiplication.
    """
    def __init__(self):
        super(Encoder, self).__init__()

    def forward(self, x, encoder_interpolation_matrix):
        """
        Forward pass for Encoder.
        Args:
            x: Input features.
            encoder_interpolation_matrix: Sparse interpolation matrix.
        Returns:
            Encoded features.
        """
        return torch.sparse.mm(encoder_interpolation_matrix, x)  # Sparse matrix multiplication

class Decoder(torch.nn.Module):
    """
    Decoder module using sparse matrix multiplication.
    """
    def __init__(self):
        super(Decoder, self).__init__()

    def forward(self, x, decoder_interpolation_matrix):
        """
        Forward pass for Decoder.
        Args:
            x: Input features.
            decoder_interpolation_matrix: Sparse interpolation matrix.
        Returns:
            Decoded features.
        """
        return torch.sparse.mm(decoder_interpolation_matrix, x)  # Sparse matrix multiplication

class IterableGcn(nn.Module):
    """
    Sequential container for a list of GCN layers.
    """
    def __init__(self, layer_list):
        super(IterableGcn, self).__init__()
        self.layers = nn.ModuleList(layer_list)  # Store layers in ModuleList

    def forward(self, x, edge_indices, edge_distances):
        """
        Forward pass through all layers.
        Args:
            x: Node features.
            edge_indices: Edge indices.
            edge_distances: Edge weights.
        Returns:
            Output tensor after all layers.
        """
        for layer in self.layers:
            x = layer(x, edge_indices, edge_distances)  # Apply each layer
        return x

class GraphAutoencoder(torch.nn.Module):
    """
    Hierarchical Graph Autoencoder with multiple pooling and unpooling stages.
    """
    def __init__(self, in_channels, 
                 out_channels,
                 n_units_input,
                 n_units_output,
                 n_layers_orig,
                 n_units_orig,
                 n_layers_pool_1,
                 n_units_pool_1,
                 n_layers_pool_2,
                 n_units_pool_2
                 ):
        super(GraphAutoencoder, self).__init__()

        self.n_units_input = n_units_input  # Number of units in input layer
        self.n_units_output = n_units_output  # Number of units in output layer
        self.n_layers_orig = n_layers_orig  # Number of original layers
        self.n_units_orig = n_units_orig  # Units per original layer
        self.n_layers_pool_1 = n_layers_pool_1  # Number of layers in first pooling
        self.n_units_pool_1 = n_units_pool_1  # Units per first pooling layer
        self.n_layers_pool_2 = n_layers_pool_2  # Number of layers in second pooling
        self.n_units_pool_2 = n_units_pool_2  # Units per second pooling layer

        # Encoder
        self.graphconv_input = GraphConv(in_channels, n_units_input).to(device)  # Input GCN layer
        self.graphconv_enc_orig = []  # List for encoder GCN layers (original space)
        self.graphconv_dec_orig = []  # List for decoder GCN layers (original space)
        for i in range(n_layers_orig):
            if i == 0:
                self.graphconv_enc_orig.append(GraphConv(n_units_input, n_units_orig[0]).to(device))  # First encoder layer
                self.graphconv_dec_orig.append(GraphConv(n_units_orig[0], n_units_output).to(device))  # First decoder layer
            else:
                self.graphconv_enc_orig.append(GraphConv(n_units_orig[i-1], n_units_orig[i]).to(device))  # Subsequent encoder layers
                self.graphconv_dec_orig.append(GraphConv(n_units_orig[i], n_units_orig[i-1]).to(device))  # Subsequent decoder layers

        self.encode = Encoder().to(device)  # First encoder (sparse)
        self.graphconv_pool_1 = []  # List for first pooling GCN layers
        self.graphconv_unpool_1 = []  # List for first unpooling GCN layers
        for i in range(n_layers_pool_1):
            if i == 0:
                self.graphconv_pool_1.append(GraphConv(n_units_orig[-1], n_units_pool_1[0]).to(device))  # First pooling layer
                self.graphconv_unpool_1.append(GraphConv(n_units_pool_1[0], n_units_orig[-1]).to(device))  # First unpooling layer
            else:
                self.graphconv_pool_1.append(GraphConv(n_units_pool_1[i-1], n_units_pool_1[i]).to(device))  # Subsequent pooling layers
                self.graphconv_unpool_1.append(GraphConv(n_units_pool_1[i], n_units_pool_1[i-1]).to(device))  # Subsequent unpooling layers

        self.encode_1 = Encoder().to(device)  # Second encoder (sparse)
        self.graphconv_pool_2 = []  # List for second pooling GCN layers
        self.graphconv_unpool_2 = []  # List for second unpooling GCN layers
        for i in range(n_layers_pool_2):
            if i == 0:
                self.graphconv_pool_2.append(GraphConv(n_units_pool_1[-1], n_units_pool_2[0]).to(device))  # First pooling layer (2nd stage)
                self.graphconv_unpool_2.append(GraphConv(n_units_pool_2[0], n_units_pool_1[-1]).to(device))  # First unpooling layer (2nd stage)
            else:
                self.graphconv_pool_2.append(GraphConv(n_units_pool_2[i-1], n_units_pool_2[i]).to(device))  # Subsequent pooling layers (2nd stage)
                self.graphconv_unpool_2.append(GraphConv(n_units_pool_2[i], n_units_pool_2[i-1]).to(device))  # Subsequent unpooling layers (2nd stage)

        # Decoder
        self.graphconv_unpool_2 = IterableGcn(list(reversed(self.graphconv_unpool_2))).to(device)  # Unpooling layers (2nd stage, reversed)
        self.decode_1 = Decoder().to(device)  # Decoder (2nd stage)
        self.graphconv_unpool_1 = IterableGcn(list(reversed(self.graphconv_unpool_1))).to(device)  # Unpooling layers (1st stage, reversed)
        self.decode = Decoder().to(device)  # Decoder (1st stage)
        self.graphconv_dec_orig = IterableGcn(list(reversed(self.graphconv_dec_orig))).to(device)  # Decoder layers (original space, reversed)

        self.graphconv_enc_orig = IterableGcn(self.graphconv_enc_orig).to(device)  # Encoder layers (original space)
        self.graphconv_pool_1 = IterableGcn(self.graphconv_pool_1).to(device)  # Pooling layers (1st stage)
        self.graphconv_pool_2 = IterableGcn(self.graphconv_pool_2).to(device)  # Pooling layers (2nd stage)

        self.out_conv_cp = GraphConv(n_units_output, out_channels, output=True).to(device)  # Output layer for cp
        self.out_conv_cfx = GraphConv(n_units_output, out_channels, output=True).to(device)  # Output layer for cfx
        self.out_conv_cfy = GraphConv(n_units_output, out_channels, output=True).to(device)  # Output layer for cfy
        self.out_conv_cfz = GraphConv(n_units_output, out_channels, output=True).to(device)  # Output layer for cfz

        self.set_requires_grad(False, [self.encode, self.encode_1, self.decode, self.decode_1])  # Freeze encoder/decoder

    def set_requires_grad(self, requires_grad, modules):
        """
        Set requires_grad for all parameters in the given modules.
        Args:
            requires_grad: Boolean flag.
            modules: List of modules.
        """
        for module in modules:
            for param in module.parameters():
                param.requires_grad = requires_grad  # Set requires_grad

    def forward(self, x, space1: Space, space2: Space):
        """
        Forward pass for the hierarchical graph autoencoder.
        Args:
            x: Input node features.
            space1: Space object for first pooling/unpooling.
            space2: Space object for second pooling/unpooling.
        Returns:
            Concatenated output tensor.
        """
        x.to(device)  # Move input to device

        # Input
        x = self.graphconv_input(x, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Input GCN

        # Original space
        x = self.graphconv_enc_orig(x, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Encoder GCNs

        # Encoder 1
        x_enc = self.encode(x, space1.encoder_sparse_interpolation.to(device))  # Sparse encoding
        x_enc = self.graphconv_pool_1(x_enc, space1.edge_indices_reduced.to(device), space1.edge_distances_reduced.to(device))  # Pooling GCNs

        # Encoder 2
        x_enc_1 = self.encode_1(x_enc, space2.encoder_sparse_interpolation.to(device))  # Sparse encoding (2nd stage)
        x_enc_1 = self.graphconv_pool_2(x_enc_1, space2.edge_indices_reduced.to(device), space2.edge_distances_reduced.to(device))  # Pooling GCNs (2nd stage)

        # Decoder 2
        x_enc_1 = self.graphconv_unpool_2(x_enc_1, space2.edge_indices_reduced.to(device), space2.edge_distances_reduced.to(device))  # Unpooling GCNs (2nd stage)
        x_dec_1 = self.decode_1(x_enc_1, space2.decoder_sparse_interpolation.to(device))  # Sparse decoding (2nd stage)

        # Decoder 1
        x_dec_1 = self.graphconv_unpool_1(x_dec_1, space1.edge_indices_reduced.to(device), space1.edge_distances_reduced.to(device))  # Unpooling GCNs (1st stage)
        x_dec = self.decode(x_dec_1, space1.decoder_sparse_interpolation.to(device))  # Sparse decoding (1st stage)

        # Original space
        x_dec = self.graphconv_dec_orig(x_dec, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Decoder GCNs (original space)

        # Output
        x_cp = self.out_conv_cp(x_dec, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Output GCN cp
        x_cfx = self.out_conv_cfx(x_dec, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Output GCN cfx
        x_cfy = self.out_conv_cfy(x_dec, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Output GCN cfy
        x_cfz = self.out_conv_cfz(x_dec, space1.edge_indices.to(device), space1.edge_distances.to(device))  # Output GCN cfz

        x_out = torch.cat((x_cp, x_cfx, x_cfy, x_cfz), dim=1)  # Concatenate outputs

        return x_out


In [ ]:
# Compute Aero Coeff for Loss

X_min = torch.tensor(scaler.data_min_, dtype=torch.float32).to(device)  # Minimum values for X features
X_max = torch.tensor(scaler.data_max_, dtype=torch.float32).to(device)  # Maximum values for X features
Y_min = torch.tensor(scalery.data_min_, dtype=torch.float32).to(device)  # Minimum values for Y features
Y_max = torch.tensor(scalery.data_max_, dtype=torch.float32).to(device)  # Maximum values for Y features

def compute_aero_coeff(ds, AoA, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor):
    """
    Compute aerodynamic moment coefficient for a given dataset.

    Args:
        ds (Tensor): Input tensor containing aerodynamic data.
        AoA (float): Angle of attack in degrees.
        X_Grid_K_Unit_Normal (Tensor): X component of grid normal.
        Y_Grid_K_Unit_Normal (Tensor): Y component of grid normal.
        Z_Grid_K_Unit_Normal (Tensor): Z component of grid normal.
        cell_area (Tensor): Area of each cell.
        coordinates_tensor (Tensor): Coordinates of each node.

    Returns:
        Tensor: Computed aerodynamic moment.
    """
    aoa = AoA * np.pi / 180.0  # Convert AoA to radians

    shear_x = ds[:, 1]  # Extract shear force in x-direction
    shear_z = ds[:, 3]  # Extract shear force in z-direction
    cp = ds[:, 0]       # Extract pressure coefficient

    taux = (shear_x - X_Grid_K_Unit_Normal * cp) / area  # Compute x-direction shear stress
    tauz = (shear_z - Z_Grid_K_Unit_Normal * cp) / area  # Compute z-direction shear stress

    my = (coordinates_tensor[:, 2] * (taux * torch.cos(aoa) + tauz * torch.sin(aoa)) - 
          (coordinates_tensor[:, 0] - xref) * (tauz * torch.cos(aoa) - taux * torch.sin(aoa))) / chord  # Compute moment arm

    moment = torch.sum(my * cell_area)  # Sum moments over all cells

    return moment

def integral_load(x, y, out, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor):
    """
    Compute the mean squared error between predicted and true aerodynamic moments.

    Args:
        x (Tensor): Input features (denormalized).
        y (Tensor): True output values (denormalized).
        out (Tensor): Predicted output values (denormalized).
        X_Grid_K_Unit_Normal (Tensor): X component of grid normal.
        Y_Grid_K_Unit_Normal (Tensor): Y component of grid normal.
        Z_Grid_K_Unit_Normal (Tensor): Z component of grid normal.
        cell_area (Tensor): Area of each cell.
        coordinates_tensor (Tensor): Coordinates of each node.

    Returns:
        Tensor: Mean squared error of the aerodynamic moment.
    """
    global X_min, X_max, Y_min, Y_max

    x_den = x - X_min / (X_max - X_min)  # Denormalize input features
    out_den = out - Y_min / (Y_max - Y_min)  # Denormalize predicted outputs
    y_den = y - Y_min / (Y_max - Y_min)  # Denormalize true outputs

    AoA = torch.tensor(0.0, dtype=torch.float32)  # Set angle of attack to zero

    moment_NN = compute_aero_coeff(out_den, AoA, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # Predicted moment
    moment_CFD = compute_aero_coeff(y_den, AoA, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # True moment

    squared_error = (moment_CFD - moment_NN) ** 2  # Compute squared error
    error_moment = torch.mean(squared_error)  # Compute mean squared error

    return error_moment


In [ ]:
def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.
    Defines the model architecture, training loop, validation, and early stopping.
    Returns the average validation loss for the current trial.
    """
    layers_orig = trial.suggest_int('layers_orig', 1, 3, step=1)  # Number of original layers
    layers_pool_1 = trial.suggest_int('layers_pool_1', 1, 3, step=1)  # Number of pool_1 layers
    layers_pool_2 = trial.suggest_int('layers_pool_2', 1, 3, step=1)  # Number of pool_2 layers

    n_units_input = trial.suggest_int('n_units_input', 32, 256, step=16)  # Input units
    n_units_output = trial.suggest_int('n_units_output', 32, 256, step=16)  # Output units
    units_orig = [trial.suggest_int(f'units_orig_{str(i+1)}', 32, 256, step=16) for i in range(int(layers_orig))]  # Units per orig layer
    units_pool_1 = [trial.suggest_int(f'units_pool_1_{str(i+1)}', 32, 384, step=16) for i in range(int(layers_pool_1))]  # Units per pool_1 layer
    units_pool_2 = [trial.suggest_int(f'units_pool_2_{str(i+1)}', 32, 512, step=16) for i in range(int(layers_pool_2))]  # Units per pool_2 layer

    model = GraphAutoencoder(
        in_channels=X_train.shape[-1],  # Number of input channels
        out_channels=1,  # Number of output channels
        n_units_input=n_units_input,  # Input units
        n_units_output=n_units_output,  # Output units
        n_layers_orig=layers_orig,  # Original layers
        n_units_orig=units_orig,  # Units per orig layer
        n_layers_pool_1=layers_pool_1,  # Pool_1 layers
        n_units_pool_1=units_pool_1,  # Units per pool_1 layer
        n_layers_pool_2=layers_pool_2,  # Pool_2 layers
        n_units_pool_2=units_pool_2  # Units per pool_2 layer
    ).to(device)  # Move model to device

    n_epochs = 500  # Number of epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
    mae_loss = nn.L1Loss()  # Mean Absolute Error loss

    scheduler = StepLR(optimizer, step_size=30, gamma=0.9)  # Learning rate scheduler

    early_stopping_patience = 100  # Early stopping patience
    best_val_loss = float("inf")  # Best validation loss
    epochs_without_improvement = 0  # Counter for early stopping
    lambda_factor = 0.01  # Weight for error moment

    for epoch in range(n_epochs):  # Training loop
        model.train()  # Set model to training mode
        total_loss = 0  # Initialize total loss
        for data in train_loader:  # Iterate over training data
            optimizer.zero_grad()  # Zero gradients
            for i in range(len(data)):  # Iterate over batch
                x, y, space, grid_data = data[i].x, data[i].y, data[i].space, data[i].grid_data  # Unpack data
                cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, coordinates_tensor = grid_data  # Unpack grid data
                out = model(x, space[0], space[1])  # Forward pass
                loss = mae_loss(out, y)  # Compute MAE loss
                error_moment = integral_load(x, y, out, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # Compute error moment
                combined_loss = loss + lambda_factor * error_moment  # Combine losses
                combined_loss.backward()  # Backpropagation
                optimizer.step()  # Optimizer step
                total_loss += combined_loss.item()  # Accumulate loss

        average_train_loss = total_loss / len(train_loader)  # Average training loss

        model.eval()  # Set model to evaluation mode
        total_val_loss = 0  # Initialize validation loss
        with torch.no_grad():  # Disable gradient computation
            for data in val_loader:  # Iterate over validation data
                for i in range(len(data)):  # Iterate over batch
                    x, y, space, grid_data = data[i].x, data[i].y, data[i].space, data[i].grid_data  # Unpack data
                    cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, coordinates_tensor = grid_data  # Unpack grid data
                    out = model(x, space[0], space[1])  # Forward pass
                    val_loss = mae_loss(out, y)  # Compute validation loss
                    error_moment_val = integral_load(x, y, out, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # Compute error moment
                    combined_loss_val = val_loss + lambda_factor * error_moment_val  # Combine losses
                    total_val_loss += combined_loss_val.item()  # Accumulate validation loss

        average_val_loss = total_val_loss / len(val_loader)  # Average validation loss

        current_lr = optimizer.param_groups[0]['lr']  # Get current learning rate
        if current_lr > 0.00005:  # Check learning rate threshold
            scheduler.step()  # Step scheduler

        if average_val_loss < best_val_loss:  # Check for improvement
            best_val_loss = average_val_loss  # Update best validation loss
            epochs_without_improvement = 0  # Reset counter
            # Model saving code can be added here if needed
        else:
            epochs_without_improvement += 1  # Increment counter

        if epochs_without_improvement >= early_stopping_patience:  # Early stopping condition
            break  # Stop training

        if 'lowest_loss' not in trial.user_attrs or average_val_loss < trial.user_attrs['lowest_loss']:  # Track lowest loss
            trial.set_user_attr('lowest_loss', average_val_loss)  # Set lowest loss

    return average_val_loss  # Return validation loss

trial_values = []  # List to store the loss values at each optimizer trial

if Execute_Train_flag:
    # Create a study object with TPE sampler (Bayesian optimization) and optimize hyperparameters
    study = optuna.create_study(
        study_name=study_name,  # Name of the study
        storage=storage_url,  # Storage URL
        direction='minimize',  # Minimize the objective
        sampler=optuna.samplers.TPESampler(seed=seed_value)  # TPE sampler with seed
    )
    study.optimize(objective, n_trials=20)  # Run optimization

    for trial in study.trials:  # Extract the lowest loss value for each trial
        trial_values.append(trial.user_attrs['lowest_loss'])  # Append loss

if Execute_Train_flag:
    # Store the loss value at each trial in a file
    with open("val_loss_vs_optimizer_trials.txt", "w") as f:  # Open file for writing
        for trial_number, loss_value in enumerate(trial_values, start=1):  # Enumerate trial values
            f.write(f"Trial {trial_number}: {loss_value}\n")  # Write to file


In [ ]:
if Transfer_Learning_flag:
    study = optuna.load_study(study_name=study_name, storage=storage_url)
# Get the best hyperparameters
best_params = study.best_params
layers_orig = best_params['layers_orig']
layers_pool_1 = best_params['layers_pool_1']
layers_pool_2 = best_params['layers_pool_2']
n_units_input = best_params['n_units_input']
n_units_output = best_params['n_units_output']
units_orig =   [best_params[f'units_orig_{str(i+1)}'] for i in range(int(layers_orig))]
units_pool_1 = [best_params[f'units_pool_1_{str(i+1)}'] for i in range(int(layers_pool_1))]
units_pool_2 = [best_params[f'units_pool_2_{str(i+1)}'] for i in range(int(layers_pool_2))]
        
model = GraphAutoencoder(in_channels=X_train.shape[-1], 
                out_channels=1,
                n_units_input = n_units_input,
                n_units_output = n_units_output,
                n_layers_orig = layers_orig,
                n_units_orig = units_orig,
                n_layers_pool_1 = layers_pool_1,
                n_units_pool_1 = units_pool_1,
                n_layers_pool_2 = layers_pool_2,
                n_units_pool_2 = units_pool_2
).to(device)

In [ ]:
def count_parameters(model):
    """
    Counts the number of trainable parameters in a PyTorch model.

    Args:
        model (torch.nn.Module): The model to count parameters for.

    Returns:
        int: Number of trainable parameters.
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)  # Sum parameters that require gradients

print('\n')
print('Compiling GCN model')

num_params = count_parameters(model)  # Count trainable parameters in the model
print(f"Number of trainable parameters in the model: {num_params} \n")

train_losses = []  # List to store training losses per epoch
val_losses = []    # List to store validation losses per epoch

# print(model)

if Execute_Train_flag:  # Check if training should be executed

    n_epochs = 2000  # Number of training epochs
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
    mae_loss = nn.L1Loss()  # Mean Absolute Error loss

    scheduler = StepLR(optimizer, step_size=30, gamma=0.9)  # Learning rate scheduler

    early_stopping_patience = 200  # Early stopping patience
    best_val_loss = float("inf")  # Initialize best validation loss
    epochs_without_improvement = 0  # Counter for early stopping
    lambda_factor = 0.01  # Weight for error moment in loss

    for epoch in range(n_epochs):  # Loop over epochs
        model.train()  # Set model to training mode
        total_loss = 0  # Accumulate training loss

        for data in train_loader:  # Iterate over training batches
            optimizer.zero_grad()  # Zero gradients

            for i in range(len(data)):  # Iterate over items in batch
                x, y, space, grid_data = data[i].x, data[i].y, data[i].space, data[i].grid_data  # Unpack data
                cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, coordinates_tensor = grid_data  # Unpack grid data
                out = model(x, space[0], space[1])  # Forward pass
                loss = mae_loss(out, y)  # Compute MAE loss
                error_moment = integral_load(x, y, out, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # Compute error moment
                combined_loss = loss + lambda_factor * error_moment  # Combine losses
                combined_loss.backward()  # Backpropagation
                optimizer.step()  # Update parameters
                total_loss += combined_loss.item()  # Accumulate loss

        average_train_loss = total_loss / len(train_loader)  # Average training loss

        # Validation
        model.eval()  # Set model to evaluation mode
        total_val_loss = 0  # Accumulate validation loss
        with torch.no_grad():  # Disable gradient computation
            for data in val_loader:  # Iterate over validation batches
                for i in range(len(data)):  # Iterate over items in batch
                    x, y, space, grid_data = data[i].x, data[i].y, data[i].space, data[i].grid_data  # Unpack data
                    cell_area, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, coordinates_tensor = grid_data  # Unpack grid data
                    out = model(x, space[0], space[1])  # Forward pass
                    val_loss = mae_loss(out, y)  # Compute MAE loss
                    error_moment_val = integral_load(x, y, out, X_Grid_K_Unit_Normal, Y_Grid_K_Unit_Normal, Z_Grid_K_Unit_Normal, cell_area, coordinates_tensor)  # Compute error moment
                    combined_loss_val = val_loss + lambda_factor * error_moment_val  # Combine losses
                    total_val_loss += combined_loss_val.item()  # Accumulate loss

        average_val_loss = total_val_loss / len(val_loader)  # Average validation loss

        print(f"Epoch {epoch + 1}, Train Loss: {average_train_loss}, Val Loss: {average_val_loss}")  # Print losses

        train_losses.append(average_train_loss)  # Store training loss
        val_losses.append(average_val_loss)  # Store validation loss

        current_lr = optimizer.param_groups[0]['lr']  # Get current learning rate
        if current_lr > 0.00005:
            scheduler.step()  # Update learning rate

        # Early stopping check
        if average_val_loss < best_val_loss:
            best_val_loss = average_val_loss  # Update best validation loss
            epochs_without_improvement = 0  # Reset counter
            model_save_path = os.path.join(model_dir, model_filename)  # Path to save model

            if os.path.exists(model_save_path):
                os.remove(model_save_path)  # Remove existing model file

            torch.save(model.state_dict(), model_save_path)  # Save model state
        else:
            epochs_without_improvement += 1  # Increment counter

        # Check for early stopping
        if epochs_without_improvement >= early_stopping_patience:
            print(f"Early stopping at {epoch + 1} after {early_stopping_patience} epochs without improvement.")  # Early stopping message
            break  # Stop training

    # Plot the training and validation loss over epochs
    plt.figure(figsize=(10, 6))  # Create figure
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')  # Plot training loss
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')  # Plot validation loss
    plt.xlabel('Epoch')  # X-axis label
    plt.ylabel('Loss (MAE)')  # Y-axis label
    plt.ylim(0, 0.1)  # Set y-axis limits
    plt.legend()  # Show legend
    plt.title('Training and Validation Loss vs Epochs')  # Plot title
    plt.grid(True)  # Show grid
    plt.show()  # Display plot

In [ ]:
def test_model_and_inverse_transform(model, test_loader, Y_test, X_test, scaler, scalery):
    """
    Test the model on the test set and inverse transform the predictions, X_test, and Y_test.

    Args:
        model (torch.nn.Module): Trained model to evaluate.
        test_loader (DataLoader): DataLoader for the test set.
        Y_test (np.ndarray): Ground truth outputs for the test set.
        X_test (np.ndarray): Input features for the test set.
        scaler (MinMaxScaler): Scaler used for X normalization.
        scalery (MinMaxScaler): Scaler used for Y normalization.

    Returns:
        tuple: (predictions, X_test_inv, Y_test_inv) - inverse transformed predictions, X_test, and Y_test.
    """
    predictions = []  # List to store model predictions
    with torch.no_grad():  # Disable gradient computation for inference
        for data in test_loader:  # Iterate over test batches
            for i in range(len(data)):  # Iterate over items in batch
                x, space = data[i].x, data[i].space  # Get input and space
                out = model(x, space[0], space[1])  # Forward pass
                predictions.append(out)  # Store prediction

    predictions = torch.cat(predictions, dim=0)  # Concatenate predictions

    samples, points, variables = Y_test.shape  # Get shape of Y_test
    predictions = np.array(predictions.cpu().numpy())  # Convert predictions to numpy array
    predictions = predictions.reshape(samples, points, variables)  # Reshape predictions

    samples, points, variables = X_test.shape  # Get shape of X_test
    X_test_reshaped = np.reshape(X_test, newshape=(-1, variables))  # Flatten X_test
    X_test_inv = scaler.inverse_transform(X_test_reshaped)  # Inverse transform X_test
    X_test_inv = np.reshape(X_test_inv, newshape=(samples, points, variables))  # Reshape X_test

    samples, points, variables = predictions.shape  # Get shape of predictions
    predictions_reshaped = np.reshape(predictions, newshape=(-1, variables))  # Flatten predictions
    predictions_inv = scalery.inverse_transform(predictions_reshaped)  # Inverse transform predictions
    predictions_inv = np.reshape(predictions_inv, newshape=(samples, points, variables))  # Reshape predictions

    samples, points, variables = Y_test.shape  # Get shape of Y_test
    Y_test_reshaped = np.reshape(Y_test, newshape=(-1, variables))  # Flatten Y_test
    Y_test_inv = scalery.inverse_transform(Y_test_reshaped)  # Inverse transform Y_test
    Y_test_inv = np.reshape(Y_test_inv, newshape=(samples, points, variables))  # Reshape Y_test

    return predictions_inv, X_test_inv, Y_test_inv  # Return denormalized arrays

if Transfer_Learning_flag:
    # Load the best model if not trained in this session
    if not Execute_Train_flag:
        model.load_state_dict(torch.load(model_dir + model_filename))  # Load model weights

    predictions_test, X_test_inv, Y_test_inv = test_model_and_inverse_transform(
        model=model,  # Trained model
        test_loader=test_loader,  # Test DataLoader
        Y_test=Y_test,  # Ground truth outputs
        X_test=X_test,  # Test inputs
        scaler=scaler,  # X scaler
        scalery=scalery  # Y scaler
    )
